### Importing libraries

In [92]:
# Importing libraries

# Data treatment
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Path
import sys
sys.path.append('../')

# Config
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

### Data loading

In [93]:
path = "../data/output/data_full.csv"

df = pd.read_csv(path)

In [94]:
# We check theres are no duplicated
df.duplicated().value_counts()

False    1026214
True          85
Name: count, dtype: int64

### Data cleaning

In [95]:
# We check important info in the dataframe to decide which columns contain relevant info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 16 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Superior Agency Code     995940 non-null   float64
 1   Superior Agency Name     667093 non-null   object 
 2   Agency Code              1001182 non-null  float64
 3   Agency Name              991412 non-null   object 
 4   Managing Unit Code       992718 non-null   float64
 5   Managing Unit Name       1006818 non-null  object 
 6   Economic Category        1007321 non-null  object 
 7   Revenue Source           987881 non-null   object 
 8   Revenue Type             994372 non-null   object 
 9   Detailing                996962 non-null   object 
 10  Updated Budgeted Amount  974984 non-null   object 
 11  Posted Amount            999880 non-null   object 
 12  Actual Amount            986834 non-null   object 
 13  Realization Percentage   1002165 non-null 

We see the following info:

* We have 1026299 entries

* Unnamed: 0 is irrelevant since it's an index, so we can remove it.

* 'Superior Agency Code' and 'Superior Agency Name' appear to refer to the same info. Code has more non-null entries so we will want to keep that info. However, since the name is more understandable for our analysis we will change the codes for their corresponding names to fill the empty names.

* 'Agency Code' and 'Agency Name' is the same case as the previous one.


#### Superior Agency

In [96]:
# We build a dataframe that gives us the code for every name. 
# We have to group by Name, convert to a dataframe and keep the Name and Code
df_superior_agency = df[['Superior Agency Code', 'Superior Agency Name']].groupby('Superior Agency Name').value_counts().reset_index()[['Superior Agency Name', 'Superior Agency Code']]

df_superior_agency.head()

,Superior Agency Name,Superior Agency Code
0,Advocacia-Geral da União,63000.0
1,Controladoria-Geral da União,37000.0
2,"Ministério da Agricultura, Pecuária e Abastec",22000.0
3,Ministério da Cidadania,55000.0
4,"Ministério da Ciência, Tecnologia, Inovações",24000.0


In [97]:
# Now we store these columns in a dictionary to rename the codes
superior_agency_dict = dict(zip(df_superior_agency['Superior Agency Code'], df_superior_agency['Superior Agency Name']))

superior_agency_dict

{63000.0: 'Advocacia-Geral da União',
 37000.0: 'Controladoria-Geral da União',
 22000.0: 'Ministério da Agricultura, Pecuária e Abastec',
 55000.0: 'Ministério da Cidadania',
 24000.0: 'Ministério da Ciência, Tecnologia, Inovações ',
 52000.0: 'Ministério da Defesa',
 25000.0: 'Ministério da Economia',
 26000.0: 'Ministério da Educação',
 39000.0: 'Ministério da Infraestrutura',
 30000.0: 'Ministério da Justiça e Segurança Pública',
 81000.0: 'Ministério da Mulher, Família e Direitos Huma',
 58000.0: 'Ministério da Pesca e Aquicultura',
 33000.0: 'Ministério da Previdência Social',
 36000.0: 'Ministério da Saúde',
 41000.0: 'Ministério das Comunicações',
 57000.0: 'Ministério das Mulheres, Igualdade Racial, da',
 35000.0: 'Ministério das Relações Exteriores',
 32000.0: 'Ministério de Minas e Energia',
 49000.0: 'Ministério do Desenvolvimento Agrário',
 53000.0: 'Ministério do Desenvolvimento Regional',
 51000.0: 'Ministério do Esporte',
 44000.0: 'Ministério do Meio Ambiente',
 38000.

We perform the replacement

In [98]:
# Replacement
df['Superior Agency Code'].replace(superior_agency_dict, inplace = True)

# But we see that we still have some names that had no code
df[df['Superior Agency Code'].isna() & df['Superior Agency Name'].notna()][['Superior Agency Code', 'Superior Agency Name']].head()

# For those values we reassign the name (only for NaN code and not NaN name)
df.loc[df['Superior Agency Code'].isna() & df['Superior Agency Name'].notna(), 'Superior Agency Code'] = df['Superior Agency Name']

/var/folders/gf/rtshnkx974q994djpf858hv40000gn/T/ipykernel_35608/2009858616.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Superior Agency Code'].replace(superior_agency_dict, inplace = True)


In [99]:
# Now Superior Agency Code is clean and we can rename it to just 'Superior Agency' and get rid of Superior Agency Name
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 16 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Superior Agency Code     1015754 non-null  object 
 1   Superior Agency Name     667093 non-null   object 
 2   Agency Code              1001182 non-null  float64
 3   Agency Name              991412 non-null   object 
 4   Managing Unit Code       992718 non-null   float64
 5   Managing Unit Name       1006818 non-null  object 
 6   Economic Category        1007321 non-null  object 
 7   Revenue Source           987881 non-null   object 
 8   Revenue Type             994372 non-null   object 
 9   Detailing                996962 non-null   object 
 10  Updated Budgeted Amount  974984 non-null   object 
 11  Posted Amount            999880 non-null   object 
 12  Actual Amount            986834 non-null   object 
 13  Realization Percentage   1002165 non-null 

In [100]:
df.rename(columns={'Superior Agency Code': 'Superior Agency'}, inplace=True)
df.drop(columns=["Superior Agency Name"], inplace = True)

In [101]:
df.sample()

,Superior Agency,Agency Code,Agency Name,Managing Unit Code,Managing Unit Name,Economic Category,Revenue Source,Revenue Type,Detailing,Updated Budgeted Amount,Posted Amount,Actual Amount,Realization Percentage,Posting Date,Fiscal Year
433779,Ministério da Economia,37904.0,Fundo do Regime Geral da Previdência Social,513001.0,COORDENACAO DE ORCAMENTO E FINANCAS DO FRGPS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MUL.P/DESCUMP.OBRIG.PREVID.ACESSORIA-PRINC.,"0,00","0,00","146,56","0,00",28/03/2018,2018


#### Agency

We can do the same with Agency

In [102]:
# We build a dataframe that gives us the code for every name. 
# We have to group by Name, convert to a dataframe and keep the Name and Code
df_agency = df[['Agency Code', 'Agency Name']].groupby('Agency Name').value_counts().reset_index()[['Agency Name', 'Agency Code']]

df_agency.head()

,Agency Name,Agency Code
0,Advocacia-Geral da União - Unidades com víncul...,63000.0
1,Agência Espacial Brasileira,20402.0
2,Agência Nacional de Aviação Civil,62201.0
3,Agência Nacional de Energia Elétrica,32210.0
4,Agência Nacional de Mineração,32396.0


In [103]:
# Now we store these columns in a dictionary to rename the codes
agency_dict = dict(zip(df_agency['Agency Code'], df_agency['Agency Name']))

agency_dict

{63000.0: 'Advocacia-Geral da União - Unidades com vínculo direto',
 20402.0: 'Agência Espacial Brasileira',
 62201.0: 'Agência Nacional de Aviação Civil',
 32210.0: 'Agência Nacional de Energia Elétrica',
 32396.0: 'Agência Nacional de Mineração',
 36213.0: 'Agência Nacional de Saúde Suplementar',
 68201.0: 'Agência Nacional de Transportes Aquaviários',
 39251.0: 'Agência Nacional de Transportes Aquaviários',
 39250.0: 'Agência Nacional de Transportes Terrestres',
 36212.0: 'Agência Nacional de Vigilância Sanitária',
 44205.0: 'Agência Nacional de Águas',
 20203.0: 'Agência Nacional do Cinema',
 32205.0: 'Agência Nacional do Petróleo, Gás Natural e Biocombustíveis',
 53205.0: 'Agência de Desenvolvimento da Amazônia',
 41231.0: 'Agêncial Nacional de Telecomunicações',
 52233.0: 'Amazônia Azul Tecnologia de Defesa S.A.',
 91214.0: 'Autoridade Pública Olímpica',
 25201.0: 'Banco Central do Brasil - Orçamento Fiscal e Seguridade Social',
 31280.0: 'Caixa de Construções de Casas para o Pes

In [104]:
# Replacement
df['Agency Code'].replace(agency_dict, inplace = True)

# But we see that we still have some names that had no code
df[df['Agency Code'].isna() & df['Agency Name'].notna()][['Agency Code', 'Agency Name']].head()

# For those values we reassign the name (only for NaN code and not NaN name)
df.loc[df['Agency Code'].isna() & df['Agency Name'].notna(), 'Agency Code'] = df['Agency Name']

/var/folders/gf/rtshnkx974q994djpf858hv40000gn/T/ipykernel_35608/373511893.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Agency Code'].replace(agency_dict, inplace = True)


In [105]:
# Now Agency Code is clean and we can rename it to just 'Agency' and get rid of Agency Name
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   Superior Agency          1015754 non-null  object 
 1   Agency Code              1025394 non-null  object 
 2   Agency Name              991412 non-null   object 
 3   Managing Unit Code       992718 non-null   float64
 4   Managing Unit Name       1006818 non-null  object 
 5   Economic Category        1007321 non-null  object 
 6   Revenue Source           987881 non-null   object 
 7   Revenue Type             994372 non-null   object 
 8   Detailing                996962 non-null   object 
 9   Updated Budgeted Amount  974984 non-null   object 
 10  Posted Amount            999880 non-null   object 
 11  Actual Amount            986834 non-null   object 
 12  Realization Percentage   1002165 non-null  object 
 13  Posting Date             1002468 non-null 

In [106]:
df.rename(columns={'Agency Code': 'Agency'}, inplace=True)
df.drop(columns=["Agency Name"], inplace = True)

In [107]:
df.sample()

,Superior Agency,Agency,Managing Unit Code,Managing Unit Name,Economic Category,Revenue Source,Revenue Type,Detailing,Updated Budgeted Amount,Posted Amount,Actual Amount,Realization Percentage,Posting Date,Fiscal Year
894712,"Ministério da Agricultura, Pecuária e Abastec","Ministério da Agricultura, Pecuária e Abasteci...",130101.0,COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA,Receitas Correntes,Receita Agropecuária,Receita Agropecuária,NaN,NaN,"0,00","80,00","0,00",30/03/2021,2021


#### Managing unit

We can do the same with Managing Unit

In [108]:
# We build a dataframe that gives us the code for every name. 
# We have to group by Name, convert to a dataframe and keep the Name and Code
df_MU = df[['Managing Unit Code', 'Managing Unit Name']].groupby('Managing Unit Name').value_counts().reset_index()[['Managing Unit Name', 'Managing Unit Code']]

df_MU.head()

,Managing Unit Name,Managing Unit Code
0,AG. NAC.DO PETROLEO GAS NAT.E BIOCOM,323030.0
1,AGENCIA DE DESENVOLVIMENTO DA AMAZONIA,533001.0
2,AGENCIA ESPACIAL BRASILEIRA/AEB,203001.0
3,AGENCIA NACIONAL DE AVIACAO CIVIL,113214.0
4,AGENCIA NACIONAL DE AVIACAO CIVIL - F. AEROV.,122002.0


In [109]:
# Now we store these columns in a dictionary to rename the codes
mu_dict = dict(zip(df_MU['Managing Unit Code'], df_MU['Managing Unit Name']))

mu_dict

{323030.0: 'AG. NAC.DO PETROLEO GAS NAT.E BIOCOM',
 533001.0: 'AGENCIA DE DESENVOLVIMENTO DA AMAZONIA',
 203001.0: 'AGENCIA ESPACIAL BRASILEIRA/AEB',
 113214.0: 'AGENCIA NACIONAL DE AVIACAO CIVIL',
 122002.0: 'AGENCIA NACIONAL DE AVIACAO CIVIL - F. AEROV.',
 323028.0: 'AGENCIA NACIONAL DE ENERGIA ELETRICA',
 253003.0: 'AGENCIA NACIONAL DE SAUDE SUPLEMENTAR',
 413001.0: 'AGENCIA NACIONAL DE TELECOMUNICACOES-SEDE',
 682010.0: 'AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS',
 393002.0: 'AGENCIA NACIONAL DE TRANSPORTES AQUAVIARIOS',
 393001.0: 'AGENCIA NACIONAL DE TRANSPORTES TERRESTRES',
 253002.0: 'AGENCIA NACIONAL DE VIGILANCIA SANITARIA',
 203003.0: 'AGENCIA NACIONAL DO CINEMA',
 913001.0: 'AUTORIDADE PUBLICA OLIMPICA',
 173057.0: 'BANCO CENTRAL DO BRASIL',
 123001.0: 'CAIXA FINANCIAMENTO IMOBILIARIO AERONAUTICA',
 154999.0: 'CAPES - DIFERENCA CAMBIAL',
 179083.0: 'CASA DA MOEDA DO BRASIL',
 153010.0: 'CENTRO FED.DE EDUC.TECNOL.CELSO S.DA FONSECA',
 153015.0: 'CENTRO FEDERAL DE EDUCACAO 

In [110]:
# Replacement
df['Managing Unit Code'].replace(mu_dict, inplace = True)

# But we see that we still have some names that had no code
print(df[df['Managing Unit Code'].isna() & df['Managing Unit Name'].notna()][['Managing Unit Code', 'Managing Unit Name']].head())
# For those values we reassign the name (only for NaN code and not NaN name)
df.loc[df['Managing Unit Code'].isna() & df['Managing Unit Name'].notna(), 'Managing Unit Code'] = df['Managing Unit Name']

/var/folders/gf/rtshnkx974q994djpf858hv40000gn/T/ipykernel_35608/3267497257.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Managing Unit Code'].replace(mu_dict, inplace = True)


   Managing Unit Code                             Managing Unit Name
9                 NaN  COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU
11                NaN             DIRETORIA DE GESTAO INTERNA/SE/CGU
35                NaN      COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA
38                NaN      COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA
94                NaN                               EMBRAPA-SETORIAL


In [111]:
# Now Managing Unit Code is clean and we can rename it to just 'MAnaging Unit' and get rid of Managing Unit Name
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   Superior Agency          1015754 non-null  object
 1   Agency                   1025394 non-null  object
 2   Managing Unit Code       1025685 non-null  object
 3   Managing Unit Name       1006818 non-null  object
 4   Economic Category        1007321 non-null  object
 5   Revenue Source           987881 non-null   object
 6   Revenue Type             994372 non-null   object
 7   Detailing                996962 non-null   object
 8   Updated Budgeted Amount  974984 non-null   object
 9   Posted Amount            999880 non-null   object
 10  Actual Amount            986834 non-null   object
 11  Realization Percentage   1002165 non-null  object
 12  Posting Date             1002468 non-null  object
 13  Fiscal Year              1026299 non-null  int64 
dtypes:

In [112]:
df.rename(columns={'Managing Code': 'Managing Unit'}, inplace=True)
df.drop(columns=["Managing Unit Name"], inplace = True)

In [113]:
df.sample()

,Superior Agency,Agency,Managing Unit Code,Economic Category,Revenue Source,Revenue Type,Detailing,Updated Budgeted Amount,Posted Amount,Actual Amount,Realization Percentage,Posting Date,Fiscal Year
533441,Ministério da Infraestrutura,Fundo Nacional de Segurança e Educação de Trân...,FUNDO NACIONAL DE SEG. E EDUCACAO DE TRANSITO,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MULTAS PREVISTAS EM LEGISL.ESPECIFICA-DIV.AT.,"0,00","0,00","78,12","0,00",13/07/2018,2018


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 13 columns):
 #   Column                   Non-Null Count    Dtype 
---  ------                   --------------    ----- 
 0   Superior Agency          1015754 non-null  object
 1   Agency                   1025394 non-null  object
 2   Managing Unit Code       1025685 non-null  object
 3   Economic Category        1007321 non-null  object
 4   Revenue Source           987881 non-null   object
 5   Revenue Type             994372 non-null   object
 6   Detailing                996962 non-null   object
 7   Updated Budgeted Amount  974984 non-null   object
 8   Posted Amount            999880 non-null   object
 9   Actual Amount            986834 non-null   object
 10  Realization Percentage   1002165 non-null  object
 11  Posting Date             1002468 non-null  object
 12  Fiscal Year              1026299 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 101.8+ MB


---

Now it's time to convert to their proper type the following columns:

* Updated Budgeted Amount (numeric)

* Posted Amount (numeric)

* Actual Amount (numeric)

* Realization Percentage (numeric)

* Posting Date (datetime)

In [115]:
categories = ['Updated Budgeted Amount', 'Posted Amount', 'Actual Amount', 'Realization Percentage', 'Posting Date']

for cat in categories:

    if cat == 'Posting Date':
        df[cat] = pd.to_datetime(df[cat], dayfirst=True)

    else:
        df[cat] = df[cat].str.replace(',', '.').astype(float).replace(0, np.nan)

We need to address some inconveniences such as:

* Replacing commas with dots in '0,00' for proper float conversion

* Replacing nan with 0 for effective data cleaning

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026299 entries, 0 to 1026298
Data columns (total 13 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   Superior Agency          1015754 non-null  object        
 1   Agency                   1025394 non-null  object        
 2   Managing Unit Code       1025685 non-null  object        
 3   Economic Category        1007321 non-null  object        
 4   Revenue Source           987881 non-null   object        
 5   Revenue Type             994372 non-null   object        
 6   Detailing                996962 non-null   object        
 7   Updated Budgeted Amount  18850 non-null    float64       
 8   Posted Amount            7042 non-null     float64       
 9   Actual Amount            966613 non-null   float64       
 10  Realization Percentage   7241 non-null     float64       
 11  Posting Date             1002468 non-null  datetime64[ns]
 12  

---

We'll analyze 'Economic Category' so we should clean values

In [117]:
df['Economic Category'].unique()

array(['Receitas Correntes', 'Receitas de Capital', nan,
       'Receitas Correntes - intra-orçamentárias', 'Sem informação',
       'Receitas de Capital - intra-orçamentárias'], dtype=object)

In [118]:
# Now we store these columns in a dictionary to rename the codes

economic_category_dict = {
    'Receitas Correntes': 'Current Revenues',
    'Receitas de Capital': 'Capital Revenues',
    'Receitas Correntes - intra-orçamentárias': 'Intra-Budgetary Current Revenues',
    'Sem informação': 'No Information',
    'Receitas de Capital - intra-orçamentárias': 'Intra-Budgetary Capital Revenues'
}

df['Economic Category'].replace(economic_category_dict, inplace = True)

/var/folders/gf/rtshnkx974q994djpf858hv40000gn/T/ipykernel_35608/1176127139.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Economic Category'].replace(economic_category_dict, inplace = True)


In [119]:
df['Economic Category'].unique()

array(['Current Revenues', 'Capital Revenues', nan,
       'Intra-Budgetary Current Revenues', 'No Information',
       'Intra-Budgetary Capital Revenues'], dtype=object)

In [120]:
# Now we should replace NaN with 'No Information
df['Economic Category'].fillna('No Information', inplace=True)

In [121]:
df['Economic Category'].unique()

array(['Current Revenues', 'Capital Revenues', 'No Information',
       'Intra-Budgetary Current Revenues',
       'Intra-Budgetary Capital Revenues'], dtype=object)

---

In [122]:
for cat in categories:

    if cat == 'Posting Date':
        # This is not numerical
        continue

    print(f'{cat}: {round((df[(df[cat].isna())].shape[0]) / (df.shape[0]) * 100, 2)}%')

Updated Budgeted Amount: 98.16%
Posted Amount: 99.31%
Actual Amount: 5.82%
Realization Percentage: 99.29%


In [123]:
# Now we save the dataframe
df.to_csv("../data/output/data_clean.csv", index = False)